In [6]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

`Ignite` is a High-level library to help with training neural networks in PyTorch. It comes with an `Engine` to setup a training loop, various metrics, handlers and a helpful contrib section! 

Below we import the following:
* **Engine**: Runs a given process_function over each batch of a dataset, emitting events as it goes.
* **Events**: Allows users to attach functions to an `Engine` to fire functions at a specific event. Eg: `EPOCH_COMPLETED`, `ITERATION_STARTED`, etc.
* **Accuracy**: Metric to calculate accuracy over a dataset, for binary, multiclass, multilabel cases. 
* **Loss**: General metric that takes a loss function as a parameter, calculate loss over a dataset.
* **RunningAverage**: General metric to attach to Engine during training. 
* **ModelCheckpoint**: Handler to checkpoint models. 
* **EarlyStopping**: Handler to stop training based on a score function. 

In [7]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage, ConfusionMatrix
from ignite.handlers import ModelCheckpoint, EarlyStopping

## dataset

In [11]:
# # transform to normalize the data
# transform = transforms.Compose([transforms.ToTensor(),
#                                 transforms.Normalize((0.5,), (0.5,))])

# # Download and load the training data
# trainset = datasets.FashionMNIST('./data', download=True, train=True, transform=transform)
# train_loader = DataLoader(trainset, batch_size=64, shuffle=True)

# # Download and load the test data
# validationset = datasets.FashionMNIST('./data', download=True, train=False, transform=transform)
# val_loader = DataLoader(validationset, batch_size=64, shuffle=True)

## simple CNN

In [9]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.convlayer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.convlayer2 = nn.Sequential(
            nn.Conv2d(32,64,3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(64*6*6,600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(600, 120)
        self.fc3 = nn.Linear(120, 10)
        
    def forward(self, x):
        x = self.convlayer1(x)
        x = self.convlayer2(x)
        x = x.view(-1,64*6*6)
        x = self.fc1(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return F.log_softmax(x,dim=1)

In [10]:
# creating model,and defining optimizer and loss
model = CNN()
# moving model to gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

In [2]:
import numpy as np

import torch
from torch import optim
import torch.nn.utils as torch_utils
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler


# Core of the library, contains an engine for training and evaluating, most of the classic machine learning metrics 
# and a variety of handlers to ease the pain of training and validation of neural networks.

from ignite.engine import Engine # Runs a given process_function over each batch of a dataset, emitting events as it goes.
from ignite.engine import Events # Events that are fired by the Engine during execution.
from ignite.metrics import RunningAverage # Compute running average of a metric or the output of process function.
from ignite.contrib.handlers.tqdm_logger import ProgressBar

from simple_nmt.utils import get_grad_norm, get_parameter_norm


VERBOSE_SILENT = 0
VERBOSE_EPOCH_WISE = 1
VERBOSE_BATCH_WISE = 2

# Gradient init -> FeedForward -> Loss -> back prop -> Gradient descent -> 현제상태 출력 // 을 만들거야. 
# Train, Valid engine두개를 선언하고
# 그 두개를 물고 있는 엔진을 하나 또 설정해
class MaximumLikelihoodEstimationEngine(Engine):

    def __init__(self, func, model, crit, optimizer, lr_scheduler, config):
        self.model = model
        self.crit = crit # criterion (loss)
        self.optimizer = optimizer
        self.lr_scheduler = lr_scheduler
        self.config = config

        super().__init__(func)
        # func을 Engine class에 보내 실행하기.. 근데 그럼 어떤 효과가 잇는거지?
        '''https://pytorch.org/ignite/_modules/ignite/engine/engine.html#Engine'''
        ''' 이해 안되면 이거 해보기.
        class adding():
            def __init__(self, a,b):
                print(a+b)
                return a+b
            
        class bdding(adding):
            def __init__(self):
                z = 1
                super(bdding, self).__init__(1,3)

        bdding()
        '''    
        

        self.best_loss = np.inf
        self.scaler = GradScaler() 


    # static : https://wikidocs.net/21054
    @staticmethod
    #@profile
    def train(engine, mini_batch):
        '''
            engine : 여기에 Engine을 받나본데? // train engine, valid engine
            mini_batch : train_loader가 return하는 객체
                pytorch.DataLoader? 이런 비슷한거 있음.
        '''
        # You have to reset the gradients of all model parameters
        # before to take another step in gradient descent.

        '''
            engine.state.iteration ; Number of iterations the engine has completed. Initialized as 0 and the first iteration is 1.
                https://pytorch.org/ignite/v0.4.6/concepts.html#state
            engine.iteration_per_update : update당 Iteration 숫자... 먼지 모르겟음.
        '''
        # accumulate gradient update
        engine.model.train() # 파라미터 학습 할 것임.
        if engine.state.iteration % engine.config.iteration_per_update == 1 or engine.config.iteration_per_update == 1: # 만약 설정을 config.iteration_per_update 을 1로 했을 경우, 매 iter마다 업데이트, 만약 설정을 10으로 하면 11마다 zero grad함.
            if engine.state.iteration > 1:
                engine.optimizer.zero_grad()

        device = next(engine.model.parameters()).device # 모델이 어느 gpu에 있는지 따오는거
        # 현재 모델이 어느 gpu에 있는지 구해서, 미니배치안에 잇는 텐서들을 해당 디바이스에 보내준다.

        mini_batch.src = (mini_batch.src[0].to(device), mini_batch.src[1]) # encoder tuple, length // 근데 tuple gpu로 옮기는데 length는 안옮기네
        mini_batch.tgt = (mini_batch.tgt[0].to(device), mini_batch.tgt[1]) # decoder x, y
            # tuple은 tensor와 length가 있음.

            # Raw target variable has both BOS and EOS token. 
            # The output of sequence-to-sequence does not have BOS token. 
            # Thus, remove BOS token for reference.
        x, y = mini_batch.src, mini_batch.tgt[0][:, 1:]
            # |x| = (batch_size, length) : x에서는 tensor, length를 가져가고
            # |y| = (batch_size, length) : y(decoder)에서는 length가 빠지고, tensor들만 들어감. 
            # encoder에는 isinstance라는게 들어가서 tuple인지 확인했었어.
            # y의 원래 텐서 모습은 {BOS, y_1, y_2, EOS}이건데, Teacher forcing답안용이기때문에 {y_1, y_2, eos} // 모델에 학습 데이터 들어갈땐 {BOS, y_1, y_2} 들어갈거야.

        with autocast(not engine.config.off_autocast):
                # Take feed-forward
                # Similar as before, the input of decoder does not have EOS token.
                # Thus, remove EOS token for decoder input.
            y_hat = engine.model(x, mini_batch.tgt[0][:, :-1])  
                # 모델에 학습 데이터 들어갈땐 {BOS, y_1, y_2} 들어갈거야.
                # |y_hat| = (b, l, |V|) : 각 미니배치별, length별, 단어별 로그 확률 값이 들어가 있음.

            loss = engine.crit(
                y_hat.contiguous().view(-1, y_hat.size(-1)), # flatten
                y.contiguous().view(-1) # flatten
            )

            backward_target = loss.div(y.size(0)).div(engine.config.iteration_per_update) 
                # NLL을 보면 -1/n *sigma(sgima)인데, Loss정의시 1/n을 안했음. 그래서 직접 나눠줘야함.
                # 그래서 미니 배치 사이즈로 나눠주고, 이터레이션 퍼 업데이트로 나눠줌.

        # gpu가 있을때 autocast가 수행
        if engine.config.gpu_id >= 0 and not engine.config.off_autocast:
            engine.scaler.scale(backward_target).backward()
        else:
            backward_target.backward()

        word_count = int(mini_batch.tgt[1].sum())
            # 단어의 갯수를 알아야 로스를 정확하게 구할 수 있음.
            # 1 : 에는 각 batch별 length가 들어가 있을 것이기 때문에
        p_norm = float(get_parameter_norm(engine.model.parameters())) # parameter norm
        g_norm = float(get_grad_norm(engine.model.parameters())) # gradient norm : 안정적일수록 작아짐.
        '''
                @torch.no_grad()
                def get_grad_norm(parameters, norm_type=2):
                    parameters = list(filter(lambda p: p.grad is not None, parameters)) # 파라미터의수

                    total_norm = 0

                    try:
                        for p in parameters:
                            total_norm += (p.grad.data**norm_type).sum() # ||L2||_norm
                        total_norm = total_norm ** (1. / norm_type) # 
                    except Exception as e:
                        print(e)

                    return total_norm


                @torch.no_grad()
                def get_parameter_norm(parameters, norm_type=2):
                    total_norm = 0

                    try:
                        for p in parameters:
                            total_norm += (p.data**norm_type).sum()
                        total_norm = total_norm ** (1. / norm_type)
                    except Exception as e:
                        print(e)

                    return total_norm
        
        '''


        if engine.state.iteration % engine.config.iteration_per_update == 0 and \
            engine.state.iteration > 0:
                # In order to avoid gradient exploding, we apply gradient clipping.
            torch_utils.clip_grad_norm_(
                engine.model.parameters(),
                engine.config.max_grad_norm,
            )

            # Take a step of gradient descent.
            # GPU가 있을 경우에, scale을 함.
            if engine.config.gpu_id >= 0 and not engine.config.off_autocast:
                # Use scaler instead of engine.optimizer.step() if using GPU.
                engine.scaler.step(engine.optimizer)
                engine.scaler.update()
            else:
                engine.optimizer.step()

            # 만약 lr 스케쥴을 한다그러면..
            # if engine.config.use_noam_decay and engine.lr_scheduler is not None:
            #     engine.lr_scheduler.step()


        loss = float(loss / word_count)
            # 단어단 로스를 구할 수 있음.
        ppl = np.exp(loss)

        return {
            'loss': loss, # 나중에 화면에 출력할 것.
            'ppl': ppl,
            '|param|': p_norm if not np.isnan(p_norm) and not np.isinf(p_norm) else 0., # 학습 초기에 None, inf뜨는 경우가 있었다. 그러면 학습이 안되므로 0을 리턴하도록한다.
            '|g_param|': g_norm if not np.isnan(g_norm) and not np.isinf(g_norm) else 0.,
        }



    @staticmethod
    def validate(engine, mini_batch):
        engine.model.eval()

        with torch.no_grad():
            device = next(engine.model.parameters()).device
            mini_batch.src = (mini_batch.src[0].to(device), mini_batch.src[1])
            mini_batch.tgt = (mini_batch.tgt[0].to(device), mini_batch.tgt[1])

            x, y = mini_batch.src, mini_batch.tgt[0][:, 1:]
            # |x| = (batch_size, length)
            # |y| = (batch_size, length)

            with autocast(not engine.config.off_autocast):
                y_hat = engine.model(x, mini_batch.tgt[0][:, :-1])
                # |y_hat| = (batch_size, n_classes)
                loss = engine.crit(
                    y_hat.contiguous().view(-1, y_hat.size(-1)),
                    y.contiguous().view(-1),
                )
        
        word_count = int(mini_batch.tgt[1].sum())
        loss = float(loss / word_count)
        ppl = np.exp(loss)

        return {
            'loss': loss,
            'ppl': ppl,
        }




    @staticmethod
    def attach(
        train_engine, validation_engine,
        training_metric_names = ['loss', 'ppl', '|param|', '|g_param|'],
        validation_metric_names = ['loss', 'ppl'],
        verbose=VERBOSE_BATCH_WISE, # 0,1,2 중 하나.
    ):
        # Attaching would be repaeted for serveral metrics.
        # Thus, we can reduce the repeated codes by using this function.
        def attach_running_average(engine, metric_name):
            RunningAverage(output_transform=lambda x: x[metric_name]).attach(
                engine,
                metric_name,
            )

        for metric_name in training_metric_names:
            attach_running_average(train_engine, metric_name)

        if verbose >= VERBOSE_BATCH_WISE:
            pbar = ProgressBar(bar_format=None, ncols=120) # progressbar만들어서 성과나오고.
            pbar.attach(train_engine, training_metric_names)

        if verbose >= VERBOSE_EPOCH_WISE:

            # 에폭이 끝날때마다 train_engine에서 print안에 있는 친구들을 출력하라고 함.
            @train_engine.on(Events.EPOCH_COMPLETED)
            def print_train_logs(engine):
                avg_p_norm = engine.state.metrics['|param|']
                avg_g_norm = engine.state.metrics['|g_param|']
                avg_loss = engine.state.metrics['loss']

                print('Epoch {} - |param|={:.2e} |g_param|={:.2e} loss={:.4e} ppl={:.2f}'.format(
                    engine.state.epoch,
                    avg_p_norm,
                    avg_g_norm,
                    avg_loss,
                    np.exp(avg_loss),
                ))

        # valid에 대해서도 동일하게 진행
        for metric_name in validation_metric_names:
            attach_running_average(validation_engine, metric_name)

        if verbose >= VERBOSE_BATCH_WISE:
            pbar = ProgressBar(bar_format=None, ncols=120)
            pbar.attach(validation_engine, validation_metric_names)

        if verbose >= VERBOSE_EPOCH_WISE:
            @validation_engine.on(Events.EPOCH_COMPLETED)
            def print_valid_logs(engine):
                avg_loss = engine.state.metrics['loss']

                print('Validation - loss={:.4e} ppl={:.2f} best_loss={:.4e} best_ppl={:.2f}'.format(
                    avg_loss,
                    np.exp(avg_loss),
                    engine.best_loss,
                    np.exp(engine.best_loss),
                ))

    @staticmethod
    def resume_training(engine, resume_epoch):
        engine.state.iteration = (resume_epoch - 1) * len(engine.state.dataloader)
        engine.state.epoch = (resume_epoch - 1)

    @staticmethod
    def check_best(engine):
        loss = float(engine.state.metrics['loss'])
        if loss <= engine.best_loss:
            engine.best_loss = loss

    @staticmethod
    def save_model(engine, train_engine, config, src_vocab, tgt_vocab):
        avg_train_loss = train_engine.state.metrics['loss']
        avg_valid_loss = engine.state.metrics['loss']

        # Set a filename for model of last epoch.
        # We need to put every information to filename, as much as possible.
        model_fn = config.model_fn.split('.')
        
        model_fn = model_fn[:-1] + ['%02d' % train_engine.state.epoch,
                                    '%.2f-%.2f' % (avg_train_loss,
                                                   np.exp(avg_train_loss)
                                                   ),
                                    '%.2f-%.2f' % (avg_valid_loss,
                                                   np.exp(avg_valid_loss)
                                                   )
                                    ] + [model_fn[-1]]

        model_fn = '.'.join(model_fn)

        # Unlike other tasks, we need to save current model, not best model.
        torch.save(
            {
                'model': engine.model.state_dict(),
                'opt': train_engine.optimizer.state_dict(),
                'config': config,
                'src_vocab': src_vocab,
                'tgt_vocab': tgt_vocab,
            }, model_fn
        )

In [3]:
# 
target_engine_class = MaximumLikelihoodEstimationEngine

In [ ]:
target_engine_class(target_engine_class.train)

In [18]:
config = {'lr_decay_start' : 0,
          'init_epoch' : 0,
          'n_epochs' : 10,
          'lr_step' : 1,
          'lr_gamma' : 0.5}

def get_scheduler(optimizer, config):
    '''Adam 같은 경우는 쓸 필요가 없고, SGD같은 경우 써야한데'''
    '''
         LR |
          1 |-----------------
            |
            |                  --
            |                      -- ... 10번부터는 lr을 반씩 줄여가면서 진행
            |_____________________________________
               1  2  3  4 ... 9 10 11 12         epochs
    
    '''
    if config['lr_step'] > 0:
        lr_scheduler = optim.lr_scheduler.MultiStepLR(
                                                    optimizer,
                                                    milestones=[i for i in range(
                                                                                max(0, config['lr_decay_start'] - 1), # 내가 원하는 시작 에폭부터
                                                                                (config['init_epoch'] - 1) + config['n_epochs'], # 내가 원하는 끝나는 에폭까지
                                                                                config['lr_step'] # 1
                                                                                )],
                                                    gamma=config['lr_gamma'], # 0.5씩 작아짐.
                                                    last_epoch=config['init_epoch'] - 1 if config['init_epoch'] > 1 else -1,
                                                )
    else:
        lr_scheduler = None

    return lr_scheduler


lr_scheduler = get_scheduler(optimizer, config)



target_engine_class(target_engine_class.train,
                    model,
                    criterion,
                    optimizer,
                    lr_scheduler,
                    config)

/Users/rainism/miniforge3/envs/tensorflow/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


In [16]:
Engine(target_engine_class.train)

In [26]:
Engine(target_engine_class.train).model

AttributeError: 'Engine' object has no attribute 'model'